*Ce notebook est distribué par Devlog sous licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions. La description complète de la license est disponible à l'adresse web http://creativecommons.org/licenses/by-nc-sa/4.0/.*

# Initiation python - Outils 2/6 : les tests

## Pourquoi écrire des tests ?

Lorsque l'on écrit un programme, il est généralement constitué de plusieurs fonctions que l'on assemble afin de décrire notre algorithme permettant de nous donner la réponse à notre problème. Un programme n'est pas forcément un développement sur un temps court. 

On voit beaucoup de librairies scientifiques qui ont plus de dix ans. Les fonctions peuvent donc être écrites à différents moments avec des échelles de temps bien différentes. On peut par exemple ajouter une fonctionnalité à un bout de code plusieurs années après en avoir écrit le cœur. 

S'il est primordial d'écrire de la documentation pour comprendre ce qui est fait, il est également judicieux d'écrire des tests pour s'assurer du bon fonctionnement de notre programme.

Il faut noter que certains types de développement logiciel s'appuient sur les tests ([Test Driven Development](http://fr.wikipedia.org/wiki/Test_Driven_Development)).

## Les types de tests

On peut citer trois types de tests primordiaux permettant de s'assurer au mieux de l'absence de bugs dans notre programme. Un programme n'est jamais à 100% sûr.

- les **tests unitaires**, 

- les **tests d'intégration**,

- les **tests du système complet**.


### Tests unitaires : niveau 0

Le but est de tester chaque petit bout de code : fonctions, méthodes, ...
   

Ces tests permettent d'être sûr que chaque brique de votre programme fonctionne correctement indépendamment des autres.
    

Néanmoins, ils ne permettent pas d'assurer le bon fonctionnement du programme dans sa globalité.


### Tests d'intégration : niveau 1

Le but est de commencer à tester de petites interactions entre les différentes unités du programme.


Ces tests peuvent être réalisés avec les mêmes outils que ceux utilisés dans les tests unitaires.


Mais il y a une différence importante: on suppose que les unités prises une à une sont valides.

### Tests du système : niveau 2

Le but est de tester le programme dans sa globalité.

On assemble à présent toutes les briques pour un problème concret.

Là encore, si les 2 premiers niveaux sont négligés, les tests du système complet ne servent à rien.

Les tests sont donc écrits à des stades différents du développement mais ont chacun leur importance. Un seul de ces trois types de tests ne suffit pas pour tester l'intégrité du programme. 

Les tests unitaires et les tests d'intégration sont généralement testés avec les mêmes outils. 

Pour le dernier type de tests, on prendra des exemples concrets d'exécution et on testera la sortie avec une solution certifiée.

## Notre cas d'étude

Nous allons calculer les coefficients de la suite de Fibonacci en utilisant les coefficients binomiaux. Les coefficients binomiaux se calculent à partir de la formule suivante :

$$
\left(
\begin{array}{c}
n \\
k
\end{array}
\right)=C_n^k=\frac{n!}{k!(n-k)!} \; \text{pour} \; k=0,\cdots,n.
$$

On en déduit alors le calcul des coefficients de la suite de Fibonacci par la formule suivante :

$$
\sum_{k=0}^n
\left(
\begin{array}{c}
n-k \\
k
\end{array} 
\right)
= F(n+1).
$$

Voici un exemple de code Python implantant cette formule :

In [ ]:
%%file ./05_tests/fibonacci.py
import numpy as np
import doctest

def factorielle(n):
    """
    calcul de n!
    
    >>> factorielle(0)
    1
    >>> factorielle(5)
    120
    
    """
    if n==1 or n==0:
        return 1
    else:
        return (n*factorielle(n-1))

def somme(deb, fin, f, fargs=()):
    """
    calcul de 
    
    $$
    \sum_{k=deb}^fin f(k, *fargs)
    $$
    
    test d'une suite arithmetique
    >>> somme(0, 10, lambda k:k)
    55.0
    
    test d'une suite geometrique
    >>> somme(1, 8, lambda k: 2**k)
    510.0

    """
    
    som = 0.
    for k in range(deb, fin + 1):
        som += f(k, *fargs)
    return som
    
def coef_binomial(n, k):
    """
    calcul de $C_n^k$
    
    >>> coef_binomial(4, 2)
    6.0
    
    """
    if k > n or k < 0:
        return 0.
    return (factorielle(n)/(factorielle(k)*factorielle(n-k)))

def fibonacci(n):
    """
    Renvoie la liste des n premiers termes de la suite de Fibonacci
    
    >>> fibonacci(10)
    [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
    
    """
    def g(k, n):
        return (coef_binomial(n - k, k))
    
    fibo = []
    for i in range(n):
        fibo.append(int(somme(0, i, g, fargs=(i,))))
    
    return fibo

if __name__ == '__main__':
    doctest.testmod(verbose=True)

On souhaite faire les tests suivants :

* **tests unitaires** : tester si les fonctions *factorielle* et *somme* fonctionnent correctement.
* **tests d'intégration** : tester si les fonctions *factorielle* et *somme* fonctionnent correctement ensemble, tester si la fonction *coef_binomial* fonctionne correctement.
* **tests du système complet** : tester si la fonction *fibonacci* donne le bon résultat.

## Les outils de tests en Python

Il existe différents outils en Python permettant de réaliser des tests ([https://wiki.python.org/moin/PythonTestingToolsTaxonomy](https://wiki.python.org/moin/PythonTestingToolsTaxonomy)). Nous nous intéresserons ici à trois d'entre eux :

- doctest
- unittest
- nosetests

## doctest

**doctest** permet de faire des tests basiques en s'appuyant par exemple sur les docstrings. 


*Rappel* : les docstrings permettent d'écrire de la documentation très facilement de nos fonctions ou de nos classes Python. Elles se placent tout de suite après une méthode, une fonction, une classe. On rappelle ici brièvement le principe en écrivant une documentation pour une fonction.

In [ ]:
def add(a, b):
    """
    addition de 2 nombres a et b
    """
    return a + b


In [ ]:
print (help(add))

**doctest** effectue :

* une recherche dans les sources des bouts de texte qui ressemblent à une session interactive Python,
* une recherche dans des fichiers texte des bouts de texte qui ressemblent à une session interactive Python,
* une exécution de ces bouts de session pour voir si le résultat est conforme.

Les sessions interactives sont représentées par le symbole **>>>**.

Pour l'utiliser, il suffit d'importer le module *doctest* et d'appeler *testmod* si on veut tester l'ensemble d'un module comme dans notre exemple *fibonacci.py*.

Voici un exemple de la sortie :

In [ ]:
!python ./05_tests/fibonacci.py

### Les directives

Il est possible d'ajouter des directives dans le texte permettant de ne pas faire le test, d'indiquer juste une partie de la sortie, ...



In [ ]:
%%file ./05_tests/fibo.txt
Voici un exemple permettant de tester le module de fibonacci :

>>> import fibonacci as f
>>> n = 30
>>> for i in range(n):
...    print (f.coef_binomial(n, i)) 
... # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
1...
145422675
155117520
145422675
...


Si **doctest** est très simple d'utilisation, on se rend bien compte qu'il est assez limité et qu'il ne permet pas de faire des tests très élaborés.

## unittest

Ce module est également appelé PyUnit et reprend l'esprit de JUnit qui permet de faire des tests en java. Il supporte :

* les tests automatiques,
* les fonctions d'initialisation et de finalisation pour chaque test,
* l'aggrégation des tests,
* l'indépendance des tests dans le rapport final.

Pour écrire des tests, il faut respecter certaines règles.

* Les tests doivent faire partie d'une classe héritée de la classe `unittest.TestCase`.
* Les noms des méthodes de cette classe doivent avoir le prefixe test pour être considérés comme tests.
* Les tests sont exécutés par ordre alphabétique.
* La fonction exécutée avant chaque test doit avoir le nom `setUp`.
* La fonction exécutée après chaque test doit avoir le nom `tearDown`.

Voici un exemple de son utilisation avec notre module *fibonacci* :

In [ ]:
%%file ./05_tests/test_fibo.py
import unittest
from fibonacci import *

class TestFibo(unittest.TestCase):
    def test_factorielle_0(self):
        self.assertEqual(factorielle(0), 1)

    def test_factorielle_5(self):
        self.assertEqual(factorielle(5), 120)
        
    def test_somme(self):
        self.assertEqual(somme(0, 10, lambda k:k), 55)

    def test_coef_binomial(self):
        self.assertEqual(coef_binomial(4, 2), 6)
        
    def test_fibo(self):
        self.assertEqual(fibonacci(10), [1, 1, 2, 3, 5, 8, 13, 21, 34, 55])
        
if __name__ == '__main__':
    unittest.main(verbosity=2) # par defaut verbosity=1

In [ ]:
!python ./05_tests/test_fibo.py

### Les assertions

Ils permettent de dire à **unittest** ce que l'on attend comme résultat du test.

- `assertEqual`: les 2 valeurs doivent être égales.
- `assertAlmostEqual`: les 2 valeurs doivent être à peu près égales.
- `assertTrue`: l'expression doit être vraie.
- `assertFalse`: l'expression doit être fausse.
- ...


In [ ]:
%%file ./05_tests/test_assert.py
import unittest

class TestAssert(unittest.TestCase):
    def test_equal(self):
        self.assertEqual(2, 1 + 1)

    def test_false(self):
        self.assertFalse(1 == 1 + 1)
        
    def test_almostEqual(self):
        self.assertAlmostEqual(0.000011, 0.000012, places=5)
        
if __name__ == '__main__':
    unittest.main(verbosity=2) # par defaut verbosity=1

In [ ]:
!python ./05_tests/test_assert.py

### Rassembler les tests

In [ ]:
%%file ./05_tests/alltests.py
import unittest

def allTests():
    from test_assert import TestAssert
    from test_fibo import TestFibo

    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(TestAssert))
    suite.addTest(unittest.makeSuite(TestFibo))

    return suite

if __name__ == '__main__':
    unittest.TextTestRunner(verbosity=2).run(allTests())

In [ ]:
!python ./05_tests/alltests.py

## nosetests

Ce module reconnaît automatiquement les tests réalisés à partir de **unittest** ou de **doctest**. Il a en plus d'autres fonctionalités intéressantes :

* tests de couverture,
* tests de profiling,
* possibilité d'ajouter des plugins,
* ...

### *Exemple d'utilisation avec doctest*

In [ ]:
!nosetests -v --with-doctest ./05_tests/fibonacci.py

### *Exemple d'utilisation avec unittest*

In [ ]:
!nosetests -v ./05_tests/test_fibo.py

### Ecriture de tests avec nosetests

**nosetests** considère qu'un fichier, un répertoire contient des tests si celui-ci satisfait l'expression régulière que l'on appellera dans la suite *matchTest*

```
((?:^|[\\b_\\.-])[Tt]est
```

En d'autres termes, il faut que les noms test ou Test soient au début du nom ou qu'ils soient précédés de - ou _.

La règle s'applique également aux fonctions, classes, ... qui se trouvent dans le fichier à tester.

### Exemples

In [ ]:
%%file ./05_tests/test_nose.py

class test_une_classe:
    def pasuntestvalide(self):
        pass

    def testvalide(self):
        pass

    def encore_un_test_valide(self):
        pass

def ouUneFonction_Test():
    pass

In [ ]:
!nosetests -v ./05_tests/test_nose.py

### Les assertions

Tout comme **unittest**, **nosetests** comprend tout un tas d'outils pour faire des assertions. Ils se trouvent dans `nose.tools`. 

Attention: contrairement à **unittest**, **nosetests** satisfait les règles de la [PEP 8](https://www.python.org/dev/peps/pep-0008#function-names). Par conséquent, `assertEqual` devient `assert_equal`.

### Initialisation et finalisation des tests

Il est possible d'exécuter du code en début et en fin de tests comme avec les fonctions `setUp` et `tearDown` de **unittest**.

### Pour les packages de tests

On peut ajouter les fonctions d'initialisation et de finalisation dans le fichier `__init__.py`. 

Les fonctions d'initialisation doivent se nommer `setup`, `setup_package`, `setUp` ou `setUpPackage`.

Les fonctions de finalisation doivent se nommer `teardown`, `teardown_package`, `tearDown` ou `tearDownPackage`.

### Pour les modules de tests

Un module de tests est un module dont le nom satisfait *matchTest*. 

De la même manière que pour les packages, les fonctions d'initialisation et de finalisation doivent avoir les noms `setup`, `setup_module`, `setUp` ou `setUpModule` et `teardown`, `teardown_module`, `tearDown` ou `tearDownModule`


### Pour les classes de tests

Une classe de tests est une classe dont le nom satisfait *matchTest*. Elle doit se trouver dans un module de tests.

Soit elle dérive de la classe `unittest.TestCase`, soit elle comporte des méthodes qui satisfont le *matchTest*.

Les fonctions d'initialisation sont `setup_class`, `setupClass`, `setUpClass`, `setupAll` ou `setUpAll`.

Les fonctions de finalisation sont `teardown_class`, `teardownClass`, `tearDownClass`, `teardownAll` ou `tearDownAll`.

### Pour les fonctions de tests

Une fonction de test est une fonction dont le nom satisfait *matchTest*. Elle doit se trouver dans un module de tests.

Contrairement aux précédents cas, il n'y a pas de format particulier pour les phases d'initialisation et de finalisation.

Il faut utiliser le décorateur `with_setup` qui se trouve dans le module `nose`.

In [ ]:
%%file ./05_tests/test_nose2.py
from nose import with_setup

def init_func():
    print ("j'inialise !!")

def end_func():
    print ("je finalise !!")
    
@with_setup(init_func, end_func)    
def test1():
    print ("test 1")


In [ ]:
!nosetests -s ./05_tests/test_nose2.py

### Les attributs

Dans **nosetests**, il est possible de sélectionner une partie des tests en utilisant des attributs.

On peut par exemple exécuter que les tests qui ne sont pas lents.

In [ ]:
%%file ./05_tests/test_nose_attr.py
from nose.plugins.attrib import attr

@attr('lent')
def test_lent():
    print ('test lent')

def test_rapide():
    print ('test rapide')

In [ ]:
!nosetests -s ./05_tests/test_nose_attr.py

In [ ]:
!nosetests -s -a '!lent' ./05_tests/test_nose_attr.py

In [ ]:
%%file ./05_tests/test_nose_attr.py
from nose.plugins.attrib import attr

@attr(vitesse='lent')
def test_lent():
    print ('test lent')

@attr(vitesse='rapide')
def test_rapide():
    print ('test rapide')

In [ ]:
!nosetests -s -a vitesse='lent' ./05_tests/test_nose_attr.py

### Les tests de couverture

Il est également possible avec **nosetests** de voir si nos tests passent bien sur l'ensemble de notre code.


In [ ]:
!nosetests --with-coverage --cover-package=fibonacci ./05_tests/test_fibo.py

On voit qu'on a oublié de tester 2 lignes de notre module. Si on creuse un peu ces 2 lignes correspondent aux lignes du *main* ce qui n'est donc pas très important.

Continuons par des exercices pratiques, avec le [notebook 05_tests_TPs](05_tests_TPs.ipynb)

In [1]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()